# Create and Submit a Slice

On FABRIC, a slice is a collection of logically-related resources representing a single execution of an experiment (or a portion of an experiment, as multiple slices may be involved). Typically, a slice represents a connected topology of resources known as slivers. Every slice is part of one and only one project.

Creating a slice can be accomplished using the FABlib library as shown in the following Jupyter notebook.

Creating any slice starts with follows three basic steps:

- Step 1: Create the slice by calling the `fablib.new_slice()` method and passing in the slice’s name.  At this point you have a slice object that you can use to build the slice.
- Step 2: Design the slice by adding nodes, networks, and other components to the slice object. The slice that you design depends on the goals of your experiment.
- Step 3: Submit the slice request. A slice is not instantiated until you submit the slice to the FABRIC service using the `slice.submit()` method. When you submit the slice request, FABRIC will try to find and allocate resources for your slice.   This step is quite complicated.  Larger slices will take longer to build and any slice might be impossible given currently available resources. While submitting a slice request there are several options that help you wait for your slice to build and handle exceptions in the build process. 



## Configure the Environment

Set the environment variable that will be used by this notebook. If you are using the FABRIC, JupyterHub some of the environment will be automatically configured for you.  You will only need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  If you do not have a bastion username and private key, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']=<INSERT_YOUR_FABRIC_PROJECT_ID>

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

### Import the FABlib API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## Option 1: Create a Slice - Basic

The easiest way to submit a slice is to call the submit method on your slice without any parameters.  By default, this will block and wait until your slice is built, management access is enabled via ssh, and all dataplane interfaces have been tested for connectivity.  In addition,  the progress of your slice’s build process will be printed.  



In [ ]:
try:
    # Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Design the slice
    node = slice.add_node(name="Node1")

    #Submit the Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Option 2: Create a Slice - Wait Silently

You can disable the progress reporting by setting `progress=False`.


In [ ]:
try:
    # Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Design the slice
    node = slice.add_node(name="Node1")

    #Submit the Request
    slice.submit(progress=False)
except Exception as e:
    print(f"Exception: {e}")

## Option 3: Create a Slice - Non-blocking Submit

You can create a slice without blockng by setting `wait=False`.  Note that if you do not wait for the slice, you will likely want to manually test your slice's state before trying to use it.  

In [ ]:
try:
    # Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Design the slice
    node = slice.add_node(name="Node1")

    # Submit the request
    slice.submit(wait=False)
    
    # Save the slice ID
    slice_id = slice.get_slice_id()
except Exception as e:
    print(f"Exception: {e}")

Test your slice's state

In [ ]:
try:
    #Get Slice
    slice = fablib.get_slice(slice_id=slice_id)

    #Wait for ssh to be active
    slice.wait_ssh(progress=True)
    
    #Run post boot config and dataplane network tests
    slice.post_boot_config()
except Exception as e:
    print(f"Exception: {e}")


## Additional Options: Timeouts and Retry Interval

For any blocking call you can set and overall `timeout` and a retry `interval`.  Both settings are in seconds.

In [ ]:
try:
    # Create a slice
    slice = fablib.new_slice(name="MySlice")

    # Design the slice
    node = slice.add_node(name="Node1")

    #Submit the Request
    slice.submit(timeout=600, interval=60)
except Exception as e:
    print(f"Exception: {e}")